In [11]:
# Imports
import sys, os
import re
import numpy as np

# For some reason everything goes to shit if you don't initialize in the PyRosetta install dir...
pyrosetta_dir = "/Users/cwbrown/src/PyRosetta.monolith.mac.release-96"
working_dir = os.getcwd()
os.chdir(pyrosetta_dir)

from rosetta import *

# Need this for PTMs
from rosetta.core.pose import add_variant_type_to_pose_residue

# Some of these don't actually get used
from toolbox import mutate_residue, cleanATOM, generate_resfile_from_pose, mutants

# Initialize Rosetta
init(extra_options="-mute core -mute basic")

# Switch back to the starting directory
os.chdir(working_dir)

# Basic Things:

1. nsAAs require special rotamer libraries that don't ship with the standard PyRosetta install. BarrickLab has
   a copy of these (see "Rotamer Libraries") that you'll need to set up in your local PyRosetta install
        
2. nsAAs can't use the standard score function supplied by the get_fa_scorefxn() method, so you need to set up
   a new score function using the 'mm_std' weights:

In [2]:
# Standard Full-Atom Score Function - SegFaults if used on nsAAs
fa_sf = get_fa_scorefxn()


# ncAA-safe score function using mm_std
mm_std_sf = create_score_function("mm_std")

# Rotamer Libraries:

1. Copy (or symlink?) nsAA rotamer libraries to:
    
        [PATH_TO_PYROSETTA]/database/rotamer/ncaa_rotlibs/ncaa_rotamer_libraries/alpha_amino_acid
            
   These currently live on UT Box in:
        
        Researchers/Brown, Colin/Rosetta/Noncanonical Rotamer Libs/ncaa_rotamer_libraries
            
2. Make sure all of the nsAA .param files are present in:
        
        [PATH_TO_PYROSETTA]/database/chemical/residue_type_sets/fa_standard/residue_types/l-ncaa
        
   They should be there in the default PyRosetta install
        
3. Make sure any nsAAs you want to use are uncommented in the residues_types.txt file:
    
        [PATH_TO_PYROSETTA]/database/chemical/residue_type_sets/fa_standard/residues_types.txt

### Use this method to mutate a single residue in a `Pose()` object to a specified ncAA

In [3]:
def mutate_ncaa(pose,residue,ncaa_name,repack=None,minimize=None):
    """
    Swap w/t AA at residue number 'residue' in 'pose' with 'ncaa_name' (3-letter code)
    
    Return a new Pose object
    
    Specify a RotamerTrialsMover for 'repack' and a MinMover for 'minimize' if desired 
        (otherwise no repacking or minimizing)
    
    Note that this assumes the ncaa .params and .rotlib files have been permanently added to the database
    """
    ncaa_pose = Pose()
    ncaa_pose.assign(pose)
    
    # Get a Residue object for the desired ncAA
    chm = core.chemical.ChemicalManager.get_instance()
    rts = chm.residue_type_set("fa_standard")
    ncaa_res = rosetta.core.conformation.ResidueFactory.create_residue( rts.name_map( ncaa_name ) )
    
    # replace the target residue with the ncAA
    ncaa_pose.replace_residue(residue,ncaa_res,orient_backbone=True)
    
    # repack and minimize
    if repack:
        repack.apply(ncaa_pose)
    if minimize:
        minimize.apply(ncaa_pose)
    
    return ncaa_pose

def make_packer_task_with_residues(pose,residues):
    """
    Builds a packer task with the specified residues activated for repacking.
    
    Did this to avoid PackerTask.temporarily_* methods which apparently we're not supposed to use
    """
    packer_task = standard_packer_task(pose)
    pack_residues = Vector1([x in residues for x in range(1,pose.n_residue())])
    packer_task.restrict_to_repacking()
    packer_task.restrict_to_residues(pack_residues)
    return packer_task



### ncAA Example: mutating residue T182 (protein coords) of TEM-1 to 3-aminotyrosine

In [4]:
# Set up MoveMap.
mm = MoveMap()
mm.set_bb(True)
mm.set_chi(True)

# Set up a Minimization mover using the mm_std score function
min_mover = MinMover()
min_mover.movemap(mm)
min_mover.score_function(mm_std_sf)
min_mover.min_type('linmin')

In [9]:
# load a PDB file (in this case TEM-1 Beta-lactamase; note that this file has already been cleaned for use in Rosetta)
pdb_filename = "1M40_ATOM.pdb"
print pdb_filename
initial_pose = pose_from_pdb(pdb_filename)

1M40_ATOM.pdb


In [10]:
# Copy the initial Pose() to use for the wild type
wt_pose = Pose()
wt_pose.assign(initial_pose)
print wt_pose.residue(157)

# mutate_ncaa makes a new Pose() object with the specified substitution; here we're using 3-aminotyrosine (code A69)
AmY157_pose = mutate_ncaa(initial_pose,157,'A69')
print AmY157_pose.residue(157)

# Should output "THR" for wt_pose and "A69" for AmY157_pose

Residue 157: THR (THR, T):
Base: THR
 Properties: POLYMER PROTEIN CANONICAL_AA SC_ORBITALS POLAR METALBINDING ALPHA_AA L_AA
 Variant types:
 Main-chain atoms:  N    CA   C  
 Backbone atoms:    N    CA   C    O    H    HA 
 Side-chain atoms:  CB   OG1  CG2  HB   HG1 1HG2 2HG2 3HG2
Atom Coordinates:
   N  : 11.735, -5.792, 27.157
   CA : 11.874, -5.576, 25.723
   C  : 10.584, -4.906, 25.246
   O  : 9.533, -5.05, 25.88
   CB : 12.122, -6.861, 24.931
   OG1: 10.931, -7.649, 25.033
   CG2: 13.356, -7.608, 25.411
   H  : 11.093, -6.285, 27.447
   HA : 12.623, -4.963, 25.569
   HB : 12.253, -6.622, 23.99
   HG1: 11.04, -8.361, 24.641
  1HG2: 13.4881, -8.51195, 24.8167
  2HG2: 14.2329, -6.97006, 25.3017
  3HG2: 13.2327, -7.8782, 26.459

Residue 157: A69 (A69, X):
Base: A69
 Properties: POLYMER PROTEIN POLAR AROMATIC ALPHA_AA L_AA
 Variant types:
 Main-chain atoms:  N    CA   C  
 Backbone atoms:    N    CA   C    O    H    HA 
 Side-chain atoms:  CB   CG   CD1  CD2  CE1  CE2  NZ1  CZ2  OT  1H

In [7]:
# Make a Pose to keep the unpacked state of the mutant
unpacked_AmY157_pose = Pose()
unpacked_AmY157_pose.assign(AmY157_pose)

# set up a RotamerTrialsMover for side-chain packing; note that we're only repacking the AmY here
task = make_packer_task_with_residues(AmY157_pose,[157,])

pack_rotamers_mover = RotamerTrialsMover(mm_std_sf, task)

pack_rotamers_mover.apply(AmY157_pose)

print "TEM-1 Wild Type: %f" % (mm_std_sf(wt_pose),)
print "AmY157 After Mutating: %f" % (mm_std_sf(unpacked_AmY157_pose),)
print "AmY157 After Packing: %f" % (mm_std_sf(AmY157_pose),)
min_mover.apply(AmY157_pose)
print "AmY157 After Packing And Minimization: %f" % (mm_std_sf(AmY157_pose),)
print "ddG: %f" % (mm_std_sf(AmY157_pose)-mm_std_sf(wt_pose))

TEM-1 Wild Type: 252.324341
AmY157 After Mutating: 832.453342
AmY157 After Packing: 613.722465
AmY157 After Packing And Minimization: 408.145269
ddG: 155.820928


### Take a gander in PyMOL - make sure to run the server script in PyMOL before you do this

In the top command line in PyMOL

    PyMOL>cd /Path/To/Your/PyRosetta/Install

    PyMOL>run PyMOLPyRosettaServer.py

In [ ]:
pymol = PyMOL_Mover()
wt_pose.pdb_info().name('Wild Type')
unpacked_AmY157_pose.pdb_info().name('AmY157_unpacked')
AmY157_pose.pdb_info().name('AmY157_PackedMinimized')
pymol.apply(wt_pose)
pymol.apply(unpacked_AmY157_pose)
pymol.apply(AmY157_pose)